In [1]:
import os
import pandas as pd
from facvae import FactorVAE
from facvae.data import shift_ret, get_dataloaders
from facvae.pipeline import test_model, train_model
from facvae.backtesting import Backtester

In [2]:
# directories
dir_main = os.path.abspath("..") + "/"
dir_code = dir_main + "code/"
dir_data = dir_main + "data/"
dir_config = dir_main + "config/"
dir_result = dir_main + "result/"

In [3]:
# constants
E = 25
B = 8
N = 74
T = 20
C = 28
H = 10
M = 32
K = 8
h_prior_size = 16
h_alpha_size = 16
h_prior_size = 16
partition = [0.7, 0.2, 0.1]
lr = 0.001
lmd = 0.0
max_grad = 0.1

In [4]:
# data
def cat(x: float) -> float:
    if x > 0.01:
        return 1.0
    elif x < -0.01:
        return -1.0
    return 0.0
    


In [5]:
df = pd.read_pickle(dir_data + "df_l1_comb.pickle")
df.sort_index(level=(0, 1), inplace=True)
df = shift_ret(df)
df

pm1m  annvol12m  annvol1m  pratio15to36w     hl52w  \
date       fsi_sid                                                           
2015-01-02 2363    -0.853006   0.657372  0.230831      -0.010812  0.142087   
           2407    -0.361725  -0.771023 -0.440008       0.241114 -0.745611   
           2578    -1.183526  -0.702803 -0.648000      -0.989553  0.397073   
           2579    -1.080143  -0.760415 -0.598295      -1.008328  0.309003   
           2587     0.348187  -0.056848  0.699870      -1.084309  1.450180   
...                      ...        ...       ...            ...       ...   
2022-12-29 5013    -0.643085   1.108016 -0.228310      -0.040278  1.277100   
           5020     0.089852  -0.946019 -0.919581       0.887375 -0.384340   
           5110     0.612618  -0.540199 -0.428574       0.879007 -0.395288   
           5117    -1.952275   1.910260  2.174899      -2.038032  2.133222   
           5721    -2.236329   0.002080  1.394505      -0.890556  0.636323   

                       90dcv  50to200prcratio  prcto52wh  prcto260dl  \
date       fsi_sid                                                     
2015-01-02 2363     0.099756        -0.473607  -0.440779   -1.025238   
           2407    -0.166680         0.190226   0.839314    0.132159   
           2578    -0.856260        -0.934151   0.032715   -0.614879   
           2579    -0.877045        -0.946264   0.145282   -0.628505   
           2587    -0.671718        -0.942724  -0.414149   -1.052401   
...                      ...              ...        ...         ...   
2022-12-29 5013    -0.003151        -0.676602  -1.341055   -0.740149   
           5020    -0.796665         0.708824   0.771973   -0.093881   
           5110    -0.420449         0.956789   0.704532    1.500790   
           5117     2.025190        -1.913336  -1.892773   -0.989403   
           5721     0.134988        -0.841576  -0.951310   -1.154222   

                    maxretpayoff  ...  5dmoneyflowvol      pm5d  \
date       fsi_sid                ...                             
2015-01-02 2363        -0.002566  ...       -1.123729 -1.113864   
           2407        -0.417013  ...        0.085790  0.109447   
           2578        -0.758530  ...       -0.575051 -0.008930   
           2579        -0.824582  ...        0.164114 -0.082312   
           2587         0.358239  ...       -0.911981 -0.857756   
...                          ...  ...             ...       ...   
2022-12-29 5013         0.115606  ...       -2.066939 -0.747724   
           5020        -0.317973  ...        0.603056  0.048445   
           5110        -0.508548  ...       -0.230054  0.223294   
           5117         1.933792  ...       -1.875464  0.020398   
           5721        -0.372165  ...       -0.646768 -0.014655   

                    logunadjprice  pm-style  vol-style  adjsto_6m  \
date       fsi_sid                                                  
2015-01-02 2363          0.251149 -0.664831  -0.701290   0.907120   
           2407          0.168957 -0.461707   0.280099  -0.411901   
           2578          0.545824 -0.647155   0.798158  -1.705545   
           2579          0.552549 -0.515808   0.832735  -0.236236   
           2587         -0.428483  0.624662   0.023684  -0.130606   
...                           ...       ...        ...        ...   
2022-12-29 5013          1.294049 -0.414217   0.189188   0.491051   
           5020          1.143512 -0.804720   1.311300  -0.678085   
           5110          0.654865 -0.294792   0.886227  -0.689597   
           5117         -2.001278  0.277436  -2.028070   0.884920   
           5721         -0.486649 -0.650150  -0.775535  -0.147960   

                    chg1yturnover    amihud  chg1yamihud       ret  
date       fsi_sid                                                  
2015-01-02 2363         -0.788316 -0.377425     1.344542  0.000385  
           2407         -0.008376 -0.503481    -0.082343 -0.003196  
           2578          1

In [6]:
# df["ret"] = df["ret"].apply(cat)
# print(df["ret"].value_counts())

In [7]:
dl_train, dl_valid, dl_test = get_dataloaders(df, T, B, partition)

In [8]:
# model
fv = FactorVAE(C, H, M, K, h_prior_size, h_alpha_size, h_prior_size).to("cuda")

# TODO: constrain the output interval
# data
def cat(x: float) -> float:
    if x > 0.01:
        return 1.0
    elif x < -0.01:
        return -1.0
    return 0.0
    
# train
train_model(fv, dl_train, lr, E, lmd=lmd, max_grad=max_grad)

# test
loss = test_model(fv, dl_test)
print("out-of-sample loss:", loss)

# predict
x, y = next(iter(dl_test))
mu_y, Sigma_y = fv.predict(x)

# backtest
len_test = next(iter(dl_test))[1].shape[0]
idx = pd.IndexSlice[df.index.get_level_values(0).unique()[-len_test:], :]
df = df.loc[idx]

df["factor"] = mu_y.flatten().cpu().numpy()
df = df[["factor", "ret"]]


bt = Backtester("factor", top_pct=0.25).feed(df).run()
bt.report()

================ Epoch 0 ================
tensor([-0.1548, -0.1433, -0.1573, -0.1659, -0.1189, -0.1312, -0.1349, -0.1554,
        -0.1522, -0.1454, -0.1343, -0.1332, -0.1389, -0.1485, -0.1256, -0.1303,
        -0.1209, -0.1367, -0.1278, -0.1307, -0.1223, -0.1420, -0.1497, -0.1211,
        -0.1253, -0.1247, -0.1488, -0.1515, -0.1441, -0.1276, -0.1307, -0.1397,
        -0.1480, -0.1276, -0.1350, -0.1183, -0.1327, -0.1244, -0.1275, -0.1215,
        -0.1418, -0.1511, -0.1226, -0.1402, -0.1279, -0.1521, -0.1553, -0.1476,
        -0.1313, -0.1332, -0.1411, -0.1394, -0.1274, -0.1348, -0.1180, -0.1298,
        -0.1243, -0.1272, -0.1213, -0.1425, -0.1508, -0.1222, -0.1401, -0.1615,
        -0.1346, -0.1329, -0.1109, -0.1128, -0.1377, -0.1694, -0.1696, -0.1882,
        -0.1428, -0.1177], device='cuda:0', grad_fn=<SelectBackward0>)
batch: 0, loss: 37.067962646484375
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
    

ValueError: Expected parameter loc (Tensor of shape (8, 74)) of distribution MultivariateNormal(loc: torch.Size([8, 74]), covariance_matrix: torch.Size([8, 74, 74])) to satisfy the constraint IndependentConstraint(Real(), 1), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [ ]:
next(iter(dl_test))[1].shape[0]

200

In [ ]:
mu_y

tensor([[-3.7231, -3.5172, -5.7296,  ..., -4.0158, -4.2369, -6.0627],
        [-4.8567, -5.5687, -5.0875,  ..., -2.9673, -2.2104, -2.6193],
        [-2.1070, -3.9919, -3.9691,  ..., -3.1521, -2.1856, -2.1630],
        ...,
        [-3.4079, -2.8614, -1.2999,  ..., -1.2931, -2.8400, -3.0919],
        [-2.3917, -2.2794, -2.8627,  ..., -0.2281,  0.0670,  0.1173],
        [-1.9779, -2.1636, -2.5198,  ..., -4.0418, -4.3209, -4.3645]],
       device='cuda:0')

In [ ]:
y

tensor([[-0.0166,  0.0130,  0.0089,  ...,  0.0535,  0.0023,  0.0325],
        [ 0.0282,  0.0135,  0.0059,  ...,  0.0553,  0.0020, -0.0018],
        [ 0.0024,  0.0078,  0.0039,  ...,  0.0524,  0.0032,  0.0516],
        ...,
        [ 0.0135,  0.0018,  0.0035,  ..., -0.0238, -0.0084, -0.0438],
        [-0.0242, -0.0162, -0.0153,  ..., -0.0516, -0.0017,  0.0105],
        [-0.0054,  0.0105,  0.0088,  ...,  0.0860,  0.0080, -0.0128]],
       device='cuda:0')

In [ ]:
mu_y, Sigma_y, mu_post, sigma_post, mu_prior, sigma_prior = fv(x, y)

In [ ]:
sigma_post

tensor([[0.6542, 0.6387, 0.6164,  ..., 0.7566, 0.7495, 0.6693],
        [0.6561, 0.6391, 0.6188,  ..., 0.7559, 0.7499, 0.6700],
        [0.6568, 0.6392, 0.6197,  ..., 0.7556, 0.7502, 0.6703],
        ...,
        [0.6601, 0.6400, 0.6238,  ..., 0.7543, 0.7509, 0.6717],
        [0.6693, 0.6421, 0.6353,  ..., 0.7510, 0.7531, 0.6754],
        [0.6573, 0.6393, 0.6202,  ..., 0.7554, 0.7502, 0.6705]],
       device='cuda:0', grad_fn=<SoftplusBackward0>)

In [ ]:
sigma_prior

tensor([[4.8794e-04, 7.8169e-01, 5.5017e+00,  ..., 3.7269e+00, 4.8551e+00,
         2.3025e+00],
        [3.9062e-04, 1.6906e+00, 7.4918e+00,  ..., 2.8093e+00, 4.3920e+00,
         2.6159e+00],
        [4.5897e-04, 1.9255e+00, 7.6841e+00,  ..., 2.5004e+00, 3.5911e+00,
         2.7195e+00],
        ...,
        [5.4078e-03, 1.1382e+00, 4.5003e+00,  ..., 2.2405e+00, 9.0304e-01,
         1.6005e+00],
        [3.2812e-04, 2.4362e+00, 8.6974e+00,  ..., 2.6522e+00, 5.6792e-01,
         2.6929e+00],
        [1.3633e-04, 3.3981e+00, 1.0480e+01,  ..., 2.6282e+00, 5.3939e-01,
         3.0180e+00]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [ ]:
import torch

def gaussian_kld(
    mu1: torch.Tensor, mu2: torch.Tensor, sigma1: torch.Tensor, sigma2: torch.Tensor
) -> torch.Tensor:
    """Calculate KL divergence of two multivariate independent Gaussian distributions

    Parameters
    ----------
    mu1 : torch.Tensor
        Means of the first Gaussian, B*K
    mu2 : torch.Tensor
        Means of the second Gaussian, B*K
    sigma1 : torch.Tensor
        Stds of the first Gaussian, B*K
    sigma2 : torch.Tensor
        Stds of the second Gaussian, B*K

    Returns
    -------
    torch.Tensor
        KL divergence, B, denoted as `kld`
    """
    kld_n = (
        torch.log(sigma2 / sigma1)
        + (sigma1**2 + (mu1 - mu2) ** 2) / sigma2**2 / 2
        - 0.5
    )
    return kld_n.sum(-1)

gaussian_kld(mu_post, mu_prior, sigma_post, sigma_prior)

tensor([4.3165e+15, 6.7759e+14, 4.1747e+13, 3.9605e+13, 2.1310e+12, 3.8314e+11,
        4.5827e+16, 1.5866e+14, 1.0213e+15,        nan,        nan, 8.8226e+04,
        5.3262e+10, 3.5303e+05, 5.8271e+03, 4.0465e+05, 4.2473e+18,        inf,
               inf,        inf,        inf, 9.2088e+35, 3.6026e+15, 3.6883e+15,
        3.5647e+14, 2.2266e+13, 1.4846e+10, 9.4447e+08, 1.0847e+08, 1.3232e+08,
        5.3063e+08, 1.2198e+09, 2.2910e+09, 2.9183e+10, 1.2810e+13, 1.2821e+14,
        5.5520e+13, 3.7567e+12, 1.7907e+09, 3.2146e+09, 2.9117e+09, 4.0268e+09,
        1.4793e+11, 3.4676e+13, 1.5649e+09, 4.8753e+09, 8.1318e+21, 6.5982e+27,
        3.2357e+28, 2.1960e+15, 4.4910e+15, 2.1220e+08, 1.1283e+06, 3.5945e+08,
        1.2586e+08, 4.1142e+14, 7.5036e+25, 7.9598e+26, 6.1532e+23, 3.2331e+20,
        9.1640e+10, 1.7823e+10, 1.9148e+09, 5.2172e+07, 1.1712e+07, 9.3568e+05,
        4.5798e+05, 1.3338e+21,        inf,        inf, 1.7879e+07, 3.5231e+25,
        9.6489e+11, 1.7431e+08, 2.6647e+

In [ ]:
mu_y

tensor([[0.0726, 0.0709, 0.0450,  ..., 0.0660, 0.0697, 0.0853],
        [0.0878, 0.0915, 0.0825,  ..., 0.0691, 0.0613, 0.0662],
        [0.0698, 0.0844, 0.0876,  ..., 0.0442, 0.0652, 0.0691],
        ...,
        [0.0700, 0.0909, 0.1109,  ..., 0.0870, 0.0551, 0.0552],
        [0.0541, 0.0588, 0.0688,  ..., 0.0995, 0.0895, 0.0877],
        [0.0554, 0.0553, 0.0552,  ..., 0.1174, 0.1153, 0.1078]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
y

tensor([[-0.0166,  0.0130,  0.0089,  ...,  0.0535,  0.0023,  0.0325],
        [ 0.0282,  0.0135,  0.0059,  ...,  0.0553,  0.0020, -0.0018],
        [ 0.0024,  0.0078,  0.0039,  ...,  0.0524,  0.0032,  0.0516],
        ...,
        [ 0.0135,  0.0018,  0.0035,  ..., -0.0238, -0.0084, -0.0438],
        [-0.0242, -0.0162, -0.0153,  ..., -0.0516, -0.0017,  0.0105],
        [-0.0054,  0.0105,  0.0088,  ...,  0.0860,  0.0080, -0.0128]],
       device='cuda:0')